# Análisis Descriptivo - Dataset CIC IIoT 2025

Análisis simplificado de detección de ataques en redes IoT siguiendo metodología CRISP-DM

## 📊 Fase 1: Comprensión del Negocio (Business Understanding)

**Objetivo**: Entender el contexto y los objetivos del análisis.

**Problema de negocio**: Las redes IoT son vulnerables a diversos tipos de ataques cibernéticos. Necesitamos identificar patrones en el tráfico de red que permitan distinguir entre operaciones normales y ataques.

**Objetivos del análisis**:
- Identificar características del tráfico que diferencian ataques de tráfico normal
- Entender los tipos de ataques presentes en el dataset
- Generar insights descriptivos para mejorar la seguridad IoT

### ⚙️ Configuración del Entorno

Configuramos el entorno y limitamos el tamaño de la muestra para análisis rápido en local.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Configuración de muestreo para análisis rápido
SAMPLE_SIZE = 5000  # Máximo de filas por clase

# Verificar directorios
if os.path.exists('data/attack_data') and os.path.exists('data/benign_data'):
    print("✓ Directorios de datos encontrados")
else:
    raise FileNotFoundError("No se encontraron los directorios de datos")

# Configurar visualizaciones
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"✓ Configuración completa - Muestra: {SAMPLE_SIZE} filas por clase")

## 🔍 Fase 2: Comprensión de los Datos (Data Understanding)

**Objetivo**: Explorar y entender la estructura, calidad y contenido de los datos.

**Pasos**:
1. Cargar una muestra de datos de ataque y benignos
2. Explorar la estructura de los datos (columnas, tipos, formas)
3. Identificar tipos de ataques presentes
4. Verificar calidad de datos (valores faltantes, duplicados)

In [ ]:
# Cargar datos - Usar la ventana de tiempo más pequeña disponible
import glob

attack_files = sorted(glob.glob('data/attack_data/attack_samples_*.csv'))
benign_files = sorted(glob.glob('data/benign_data/benign_samples_*.csv'))

def extract_time_window(filename):
    match = re.search(r'(\d+)sec', filename)
    return int(match.group(1)) if match else 999

# Encontrar ventana de tiempo común más pequeña
attack_times = {extract_time_window(f): f for f in attack_files}
benign_times = {extract_time_window(f): f for f in benign_files}
common_times = sorted(set(attack_times.keys()) & set(benign_times.keys()))

if common_times:
    time_window = common_times[0]
    attack_file = attack_times[time_window]
    benign_file = benign_times[time_window]
    
    print(f"📁 Usando ventana de tiempo: {time_window} segundos")
    print(f"   - Ataque: {os.path.basename(attack_file)}")
    print(f"   - Benigno: {os.path.basename(benign_file)}")
    
    # Cargar muestra de datos
    attack_df = pd.read_csv(attack_file, nrows=SAMPLE_SIZE)
    benign_df = pd.read_csv(benign_file, nrows=SAMPLE_SIZE)
    
    print(f"\n✓ Datos cargados:")
    print(f"   - Ataque: {len(attack_df)} muestras")
    print(f"   - Benigno: {len(benign_df)} muestras")
else:
    raise FileNotFoundError("No se encontraron ventanas de tiempo comunes")

In [ ]:
# Explorar estructura de los datos
print("=" * 60)
print("ESTRUCTURA DE LOS DATOS")
print("=" * 60)

print(f"\n📊 Forma de los DataFrames:")
print(f"   - Ataque: {attack_df.shape} (filas, columnas)")
print(f"   - Benigno: {benign_df.shape} (filas, columnas)")

print(f"\n📋 Columnas totales: {len(attack_df.columns)}")
print(f"\n🔍 Primeras 10 columnas:")
print(list(attack_df.columns[:10]))

print(f"\n📝 Tipos de datos:")
print(attack_df.dtypes.value_counts())

print(f"\n👀 Primeras filas de datos de ataque:")
display(attack_df.head(3))

In [ ]:
# Analizar tipos de ataques y calidad de datos
print("=" * 60)
print("TIPOS DE ATAQUES")
print("=" * 60)

if 'label2' in attack_df.columns:
    print(f"\n📊 Categorías de ataque (label2):")
    print(attack_df['label2'].value_counts())

if 'label3' in attack_df.columns:
    print(f"\n📊 Tipos específicos de ataque (label3):")
    print(attack_df['label3'].value_counts())

print(f"\n" + "=" * 60)
print("CALIDAD DE DATOS")
print("=" * 60)

print(f"\n🔍 Valores faltantes:")
print(f"   - Ataque: {attack_df.isnull().sum().sum()} valores faltantes")
print(f"   - Benigno: {benign_df.isnull().sum().sum()} valores faltantes")

print(f"\n🔍 Duplicados:")
print(f"   - Ataque: {attack_df.duplicated().sum()} filas duplicadas")
print(f"   - Benigno: {benign_df.duplicated().sum()} filas duplicadas")

## 🛠️ Fase 3: Preparación de los Datos (Data Preparation)

**Objetivo**: Preparar los datos para el análisis descriptivo.

**Pasos**:
1. Combinar datos de ataque y benignos
2. Crear variable objetivo (attack/benign)
3. Seleccionar características numéricas relevantes
4. Limpiar datos (valores faltantes, infinitos)

In [ ]:
# Combinar y preparar datos
print("📦 Preparando datos para análisis...")

# Combinar DataFrames
combined_df = pd.concat([attack_df, benign_df], ignore_index=True)

# Crear variable objetivo
combined_df['target'] = combined_df['label1']  # attack o benign

# Seleccionar solo características numéricas (excluyendo columnas de identificación)
exclude_cols = ['device_name', 'device_mac', 'label_full', 'label1', 'label2', 'label3', 'label4', 
                'timestamp', 'timestamp_start', 'timestamp_end', 'target']
numeric_cols = combined_df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"✓ Características numéricas seleccionadas: {len(feature_cols)}")
print(f"✓ Total de muestras: {len(combined_df)}")
print(f"   - Ataque: {(combined_df['target'] == 'attack').sum()}")
print(f"   - Benigno: {(combined_df['target'] == 'benign').sum()}")

# Limpiar valores infinitos
X = combined_df[feature_cols].copy()
X = X.replace([np.inf, -np.inf], np.nan)

# Imputar valores faltantes con mediana
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)

y = combined_df['target']

print(f"\n✓ Datos preparados: X shape {X.shape}, y shape {y.shape}")

## 📈 Fase 4: Análisis Descriptivo (Modeling)

**Objetivo**: Realizar análisis estadísticos y visualizaciones para describir los datos.

**Pasos**:
1. Distribución de clases (ataque vs benigno)
2. Estadísticas descriptivas por clase
3. Visualización de características clave
4. Análisis de diferencias entre clases

In [ ]:
# 1. Distribución de clases
print("=" * 60)
print("DISTRIBUCIÓN DE CLASES")
print("=" * 60)

class_dist = y.value_counts()
print(f"\n{class_dist}")
print(f"\nProporción:")
print(f"   - Ataque: {class_dist['attack']/len(y)*100:.1f}%")
print(f"   - Benigno: {class_dist['benign']/len(y)*100:.1f}%")

# Visualización
plt.figure(figsize=(8, 5))
sns.countplot(x=y, palette=['#e74c3c', '#3498db'])
plt.title('Distribución de Tráfico: Ataque vs Benigno', fontsize=14, fontweight='bold')
plt.xlabel('Tipo de Tráfico', fontsize=12)
plt.ylabel('Cantidad de Muestras', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 2. Estadísticas descriptivas por clase
print("=" * 60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 60)

# Seleccionar algunas características clave para análisis
# Buscar columnas relacionadas con paquetes, duración, etc.
key_features = []
for col in X.columns:
    if any(keyword in col.lower() for keyword in ['packet', 'duration', 'length', 'total', 'count']):
        key_features.append(col)
        if len(key_features) >= 5:  # Limitar a 5 características
            break

if len(key_features) < 5:
    key_features = X.columns[:5].tolist()  # Si no hay suficientes, usar las primeras

print(f"\n📊 Analizando {len(key_features)} características clave:")
for feat in key_features:
    print(f"   - {feat}")

# Estadísticas por clase
print(f"\n📈 Estadísticas descriptivas (Ataque):")
attack_stats = X[y == 'attack'][key_features].describe()
display(attack_stats)

print(f"\n📈 Estadísticas descriptivas (Benigno):")
benign_stats = X[y == 'benign'][key_features].describe()
display(benign_stats)

In [ ]:
# 3. Visualización de características clave
print("=" * 60)
print("VISUALIZACIÓN DE CARACTERÍSTICAS")
print("=" * 60)

# Crear visualizaciones comparativas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(key_features[:6]):
    ax = axes[idx]
    
    # Box plot comparativo
    attack_data = X[y == 'attack'][feature].dropna()
    benign_data = X[y == 'benign'][feature].dropna()
    
    ax.boxplot([attack_data, benign_data], labels=['Ataque', 'Benigno'], patch_artist=True,
               boxprops=dict(facecolor='lightblue', alpha=0.7),
               medianprops=dict(color='red', linewidth=2))
    ax.set_title(f'{feature}', fontweight='bold')
    ax.set_ylabel('Valor')
    ax.grid(True, alpha=0.3)

plt.suptitle('Comparación de Características: Ataque vs Benigno', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Interpretación: Los box plots muestran las diferencias en distribución de características entre ataques y tráfico benigno.")

In [ ]:
# 4. Análisis de diferencias significativas
print("=" * 60)
print("ANÁLISIS DE DIFERENCIAS")
print("=" * 60)

from scipy import stats

print(f"\n📊 Comparación de medias (Ataque vs Benigno):\n")

differences = []
for feature in key_features:
    attack_vals = X[y == 'attack'][feature].dropna()
    benign_vals = X[y == 'benign'][feature].dropna()
    
    if len(attack_vals) > 0 and len(benign_vals) > 0:
        attack_mean = attack_vals.mean()
        benign_mean = benign_vals.mean()
        diff_pct = ((attack_mean - benign_mean) / benign_mean * 100) if benign_mean != 0 else 0
        
        differences.append({
            'Característica': feature,
            'Media Ataque': attack_mean,
            'Media Benigno': benign_mean,
            'Diferencia %': diff_pct
        })

diff_df = pd.DataFrame(differences)
diff_df = diff_df.sort_values('Diferencia %', key=abs, ascending=False)
display(diff_df)

print(f"\n✅ Características con mayor diferencia entre clases:")
print(f"   - {diff_df.iloc[0]['Característica']}: {diff_df.iloc[0]['Diferencia %']:.1f}%")
print(f"   - {diff_df.iloc[1]['Característica']}: {diff_df.iloc[1]['Diferencia %']:.1f}%")

## 📊 Fase 5: Evaluación e Insights (Evaluation)

**Objetivo**: Resumir hallazgos clave y generar insights para la toma de decisiones.

**Pasos**:
1. Resumir diferencias clave entre ataques y tráfico benigno
2. Identificar características más discriminativas
3. Generar recomendaciones basadas en los hallazgos

In [ ]:
# Resumen de hallazgos
print("=" * 60)
print("RESUMEN DE HALLAZGOS")
print("=" * 60)

print(f"\n📋 Datos analizados:")
print(f"   - Total de muestras: {len(combined_df)}")
print(f"   - Ataques: {(y == 'attack').sum()} ({(y == 'attack').sum()/len(y)*100:.1f}%)")
print(f"   - Benigno: {(y == 'benign').sum()} ({(y == 'benign').sum()/len(y)*100:.1f}%)")
print(f"   - Características analizadas: {len(feature_cols)}")

print(f"\n🔍 Características más discriminativas:")
if 'diff_df' in locals():
    top_features = diff_df.head(3)
    for idx, row in top_features.iterrows():
        print(f"   {idx+1}. {row['Característica']}: {row['Diferencia %']:.1f}% de diferencia")

print(f"\n💡 Insights clave:")
print(f"   1. Se identificaron diferencias significativas en las características analizadas")
print(f"   2. Las características relacionadas con paquetes, duración y longitud muestran patrones distintos")
print(f"   3. Estas diferencias pueden ser utilizadas para detectar ataques en tiempo real")

print(f"\n✅ Recomendaciones:")
print(f"   1. Monitorear las características identificadas como más discriminativas")
print(f"   2. Implementar umbrales de alerta basados en los valores anómalos observados")
print(f"   3. Considerar estas características para modelos de machine learning de detección")

## 🚀 Fase 6: Conclusiones (Deployment)

**Resumen del análisis**:

Este análisis descriptivo ha permitido:

1. **Entender la estructura de los datos**: Identificamos los tipos de ataques presentes y la distribución de clases

2. **Identificar características clave**: Encontramos características que muestran diferencias significativas entre ataques y tráfico benigno

3. **Generar insights accionables**: Las diferencias identificadas pueden ser utilizadas para mejorar la detección de ataques en redes IoT

**Próximos pasos sugeridos**:
- Implementar modelos de clasificación usando las características identificadas
- Desarrollar sistema de monitoreo en tiempo real
- Validar los hallazgos con datos adicionales